In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import datetime

In [2]:
train = pd.read_csv("train.csv")

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108129 entries, 0 to 108128
Data columns (total 14 columns):
project_id               108129 non-null object
name                     108126 non-null object
desc                     108120 non-null object
goal                     108129 non-null float64
keywords                 108129 non-null object
disable_communication    108129 non-null bool
country                  108129 non-null object
currency                 108129 non-null object
deadline                 108129 non-null int64
state_changed_at         108129 non-null int64
created_at               108129 non-null int64
launched_at              108129 non-null int64
backers_count            108129 non-null int64
final_status             108129 non-null int64
dtypes: bool(1), float64(1), int64(6), object(6)
memory usage: 10.8+ MB


In [4]:
train.loc[train['desc'].isnull(),'desc'] = train['keywords'].map(lambda x:re.sub(r"-"," ",x))

In [5]:
col_date=['state_changed_at','created_at','launched_at','deadline']

for i in col_date:
    train[i] =  pd.to_datetime(train[i])

In [6]:
train["desc_len"] = train["desc"].map(lambda x: len(x.split(" ")))

In [7]:
train['launched_b_deadline']=(train["deadline"]-train['launched_at']).map(lambda x: x.microseconds)
train['created_b_deadline']=(train["deadline"]-train['created_at']).map(lambda x: x.microseconds)
train['state_a_deadline']=(train["state_changed_at"]-train['deadline']).map(lambda x: x.microseconds)

In [8]:
train.head()

,project_id,name,desc,goal,keywords,disable_communication,country,currency,deadline,state_changed_at,created_at,launched_at,backers_count,final_status,desc_len,launched_b_deadline,created_b_deadline,state_a_deadline
0,kkst1451568084,drawing for dollars,I like drawing pictures. and then i color them...,20.0,drawing-for-dollars,False,US,USD,1970-01-01 00:00:01.241333999,1970-01-01 00:00:01.241334017,1970-01-01 00:00:01.240600507,1970-01-01 00:00:01.240602723,3,1,26,731,733,0
1,kkst1474482071,Sponsor Dereck Blackburn (Lostwars) Artist in ...,"I, Dereck Blackburn will be taking upon an inc...",300.0,sponsor-dereck-blackburn-lostwars-artist-in-re...,False,US,USD,1970-01-01 00:00:01.242429000,1970-01-01 00:00:01.242432018,1970-01-01 00:00:01.240960224,1970-01-01 00:00:01.240975592,2,0,25,1453,1468,3
2,kkst183622197,Mr. Squiggles,So I saw darkpony's successfully funded drawin...,30.0,mr-squiggles,False,US,USD,1970-01-01 00:00:01.243027560,1970-01-01 00:00:01.243027818,1970-01-01 00:00:01.242163613,1970-01-01 00:00:01.242164398,0,0,22,863,863,0
3,kkst597742710,Help me write my second novel.,Do your part to help out starving artists and ...,500.0,help-me-write-my-second-novel,False,US,USD,1970-01-01 00:00:01.243555740,1970-01-01 00:00:01.243556121,1970-01-01 00:00:01.240963795,1970-01-01 00:00:01.240966730,18,1,23,2589,2591,0
4,kkst1913131122,Support casting my sculpture in bronze,"I'm nearing completion on a sculpture, current...",2000.0,support-casting-my-sculpture-in-bronze,False,US,USD,1970-01-01 00:00:01.243769880,1970-01-01 00:00:01.243770317,1970-01-01 00:00:01.241177914,1970-01-01 00:00:01.241180541,1,0,18,2589,2591,0


In [9]:
ignore_features = ["final_status",'project_id', 'name', 'desc', 'keywords']+col_date
features = list(train.columns)
features = [i for i in features if i not in ignore_features] 
label = "final_status"

In [10]:
X = train[features].values
y = train[label].values

In [11]:
print(X[0,:])
print(y[0])

[20.0 False 'US' 'USD' 3 26 731 733 0]
1


In [12]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
#converts values from 0 to n-1
labelencoder_X_1 = LabelEncoder()
X[:,1] = labelencoder_X_1.fit_transform(X[:,1])

labelencoder_X_2 = LabelEncoder()
X[:,2] = labelencoder_X_2.fit_transform(X[:,2])

labelencoder_X_3 = LabelEncoder()
X[:,3] = labelencoder_X_3.fit_transform(X[:,3])

In [13]:
#convert to sparse matrix of each feature.
#categorical features: which to colmns to be treated for converting to sparse.
onehotencoder = OneHotEncoder(categorical_features = [1,2,3])
X = onehotencoder.fit_transform(X).toarray()

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [15]:
print(X_train.shape, X_test.shape)

(86503, 28) (21626, 28)


In [16]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

In [17]:
import warnings
warnings.filterwarnings('ignore')
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras import optimizers
#The simplest type of model is the Sequential model, a linear stack of layers. 

Using TensorFlow backend.


In [18]:
classifier = Sequential()

#Adding the input layer and the first hidden layer with dropout
classifier.add(Dense(activation="selu", kernel_initializer="uniform", input_dim=X_train.shape[1], units=X_train.shape[1]//2))
classifier.add(Dropout(rate = 0.2))

#Adding the second hidden layer
classifier.add(Dense(activation="selu", kernel_initializer="uniform", units=X_train.shape[1]//2))
classifier.add(Dropout(rate = 0.2))

#Adding the third hidden layer
classifier.add(Dense(activation="selu", kernel_initializer="uniform", units=6))
classifier.add(Dropout(rate = 0.1))

#Adding the output layer
classifier.add(Dense(activation="sigmoid", kernel_initializer="uniform", units=1))

#Compiling the ANN
#sgd = optimizers.SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
classifier.compile(optimizer = 'adam', loss = "binary_crossentropy", metrics=["accuracy"])

In [19]:
classifier.fit(X_train, y_train, batch_size=50, epochs=50)

Epoch 1/50
86503/86503 [==============================] - 4s 49us/step - loss: 0.4719 - acc: 0.7805
Epoch 2/50
86503/86503 [==============================] - 4s 44us/step - loss: 0.3938 - acc: 0.8132
Epoch 3/50
86503/86503 [==============================] - 4s 47us/step - loss: 0.3750 - acc: 0.8187
Epoch 4/50
86503/86503 [==============================] - 4s 46us/step - loss: 0.3671 - acc: 0.8215
Epoch 5/50
86503/86503 [==============================] - 4s 46us/step - loss: 0.3575 - acc: 0.8287
Epoch 6/50
86503/86503 [==============================] - 4s 45us/step - loss: 0.3552 - acc: 0.8298
Epoch 7/50
86503/86503 [==============================] - 4s 43us/step - loss: 0.3520 - acc: 0.8316
Epoch 8/50
86503/86503 [==============================] - 4s 46us/step - loss: 0.3503 - acc: 0.8336
Epoch 9/50
86503/86503 [==============================] - 4s 43us/step - loss: 0.3452 - acc: 0.8375
Epoch 10/50
86503/86503 [==============================] - 4s 43us/step - loss: 0.3413 - acc: 0.8407

In [20]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

In [21]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred, target_names=['0','1']))

             precision    recall  f1-score   support

          0       0.79      0.95      0.86     14824
          1       0.82      0.44      0.57      6802

avg / total       0.80      0.79      0.77     21626

